#1. Below cell contains some constants



In [ ]:
DIM_X = 300
DIM_W = 300
DIM_H = 400
DIM_RATING = 6

LEN_Y = 20 + 1

LVT_DICT_SIZE = 8000

W_EOS = "<eos>"
W_UNK = "<unk>"
W_START = "<start>"

MIN_SUM_LEN = 5

import nltk
nltk.download('punkt')

#2. This cell is for preprocessing data change appropriate paths and corpus size

In [ ]:
#@title
# -*- coding: utf-8 -*-
#pylint: skip-file
import sys
import os
import numpy as np
import gzip
import pickle
import json
import string
import argparse
import time
import datetime
#from commons import *
import nltk
import operator
from nltk.tokenize import sent_tokenize

stop_words_f = "/content/drive/MyDrive/data/stopwords_en.txt"

def load_stop_words():
    stop_words = {}
    with open(stop_words_f) as f:
        for line in f:
            line = line.strip("\n")
            stop_words[line] = 1
    return stop_words

def load_amazon():

    LFW_T = 20 #3 for music, 20 for others

    amazon_all = "./data/amazon_review/aggressive_dedup.json"
    amazon_mini = "./data/amazon_review/mini.json"
    amazon_core5 = "./data/amazon_review/kcore_5.json"
    amazon_movie = "./data/amazon_review/item_cats/reviews_Movies_and_TV_5.json"
    amazon_music = './data/Musical_Instruments_5.json'
    amazon_data = '/content/drive/MyDrive/data/reviews_Electronics_5.json'

    size_corpus = 1400000 #for electronics :1,689,188 . #for books :8,898,041. #82836502.    
    stop_words = load_stop_words()

    dic = {}
    i2w = {}
    w2i = {}
    i2user = {}
    user2i = {}
    i2item = {}
    item2i = {}
    user2w = {}
    item2w = {}
    x_raw = []
    timestamps = []
    w2df = {}
 
    translator = str.maketrans('', '', string.punctuation) 
    with open(amazon_data) as f:
        i = 0.
        for line in f:
            try:
                line = line.strip("\n")
                a = json.loads(line)
                user_id = a["reviewerID"]
                item_id = a["asin"]
                review = a["reviewText"].lower()
                rating = a["overall"]
                summary = a["summary"].lower()
                unix_time = a["unixReviewTime"]
                raw_time = a["reviewTime"]

                if user_id not in user2i:
                    user2i[user_id] = len(i2user)
                    i2user[user2i[user_id]] = user_id
                if item_id not in item2i:
                    item2i[item_id] = len(i2item)
                    i2item[item2i[item_id]] = item_id
                
                if len(summary.split()) < MIN_SUM_LEN:
                    rev_sents = sent_tokenize(review)
                    for sent in rev_sents:
                        summary = sent
                        if len(summary.split()) >= MIN_SUM_LEN:
                            break
                
                if len(summary.split()) < MIN_SUM_LEN:
                    continue

                review = review.translate(translator)
                terms = review.split()
                review_words = []
                for t in terms:
                    if t in stop_words:
                        continue
                    review_words.append(t)
                    if t in dic:
                        dic[t] += 1
                    else:
                        dic[t] = 1
                    
                    if t in w2df:
                        w2df[t].add(len(timestamps))
                    else:
                        w2df[t] = set([item_id])

                summary = summary.translate(translator) 
                terms = summary.split() 
                for t in terms:
                    if t in dic:
                        dic[t] += 1
                    else:
                        dic[t] = 1

                    if t in w2df:
                        w2df[t].add(len(timestamps))
                    else:
                        w2df[t] = set([item_id])

                x_raw.append([user2i[user_id], item2i[item_id], rating, terms, review_words, unix_time, raw_time])
                timestamps.append(unix_time)
            except KeyError:
                print ("ops: " + line)
            i += 1.
            print ('\r{0}'.format(i / size_corpus) + " / 1 ",)
            if i>1400000:
              break
    
    timestamps.sort()
    spliter1 = timestamps[int(len(timestamps) * 0.8)]
    spliter2 = timestamps[int(len(timestamps) * 0.9)]
    x_raw_train = []
    x_raw_test = []
    x_raw_dev = []
    train_uid = {}
    train_iid = {}
    for i in range(len(x_raw)):
        if x_raw[i][5] < spliter1:
            x_raw_train.append(x_raw[i])
            train_uid[x_raw[i][0]] = 1
            train_iid[x_raw[i][1]] = 1
        elif x_raw[i][5] < spliter2:
            # test samples must be in train set
            if (x_raw[i][0] in train_uid) and (x_raw[i][1] in train_iid):
                x_raw_dev.append(x_raw[i])
        else:
            if (x_raw[i][0] in train_uid) and (x_raw[i][1] in train_iid):
                x_raw_test.append(x_raw[i])
    

    new_dic = {}
    w2i = {}
    iw2 = {}
    w2w = {}
    hfw = []
    new_w2df = {}
    aq= [w2df[i]  for i in w2df if len(w2df[i])==1]
    print(aq)
    if W_EOS not in dic:
        dic[W_EOS] = len(dic)
        w2df[W_EOS] = set([1])
    if W_UNK not in dic:
        dic[W_UNK] = len(dic)
        w2df[W_UNK] = set([1])

    for w in dic:
        if dic[w] >= LFW_T:
            w2i[w] = len(new_dic)
            i2w[w2i[w]] = w
            new_dic[w] = dic[w]
            w2w[w] = w
            new_w2df[w] = np.log10(len(timestamps) / len(w2df[w])) 
        else:
            w2w[w] = W_UNK

    new_w2df[W_EOS] = 1e-15
    new_w2df[W_UNK] = 1e-15

    sorted_x = sorted(new_dic.items(), key=operator.itemgetter(1), reverse=True)
    for w in sorted_x:
        hfw.append(w[0])

    for data_item in x_raw_train:
        [uid, iid, _, summ, review_words, _, _] = data_item
        for w in (summ + review_words):
            if uid in user2w:
                user2w[uid].add(w2i[w2w[w]])
            else:
                user2w[uid] = {w2i[w2w[w]]}
            if iid in item2w:
                item2w[iid].add(w2i[w2w[w]])
            else:
                item2w[iid] = {w2i[w2w[w]]}

    all_data = [x_raw_train, x_raw_dev, x_raw_test, dic, new_dic, hfw, i2w, w2i, w2w, new_w2df, i2user, user2i, i2item, item2i, user2w, item2w]

    o = open('/content/drive/MyDrive/data/data_amazon_electronics.pkl', 'wb')
    pickle.dump(all_data, o, protocol = pickle.HIGHEST_PROTOCOL)
    o.close()
    
    print (len(dic), len(new_dic), len(i2w), len(w2i), len(w2w))
    print ("#dic=", len(dic), "#new_dic=", len(new_dic), "#user=", len(i2user), "#item=", len(i2item))
    print ("#x_raw=", len(x_raw), "#x_train=", len(x_raw_train),"#x_dev=", len(x_raw_dev),  "#x_test=", len(x_raw_test))
    x_raw = []

    ###################################################
    # movie
    #dic= 2719934 #user= 123960 #item= 50052
    #x_raw= 1697533 #x_train= 1357893 #x_test= 163529
    ###################################################
    # kcore-5
    #dic= 17511889 #user= 3035045 #item= 1569973
    #x_raw= 41135696 #x_train= 32906330 #x_test= 4928037
    ###################################################


if __name__ == "__main__":
    #parser = argparse.ArgumentParser()
    #parser.add_argument("data", help="which dataset will be processed")
    #args = parser.parse_args()
    data = "amazon"
    if data == "amazon":
        load_amazon()
    elif data == "yelp":
        load_yelp()
    elif data == "yelp5":
        load_yelp5()
    else:
        print ("error: data")
    print(MIN_SUM_LEN)


#3 Initialise modules function

In [ ]:
import sys
import  pickle
#from nrec import *
#from utils_pg import *
#import data as datar
import math
import time
import argparse
#from commons import *
import copy
import tensorflow as tf
#from os import makedirs
#from os.path import exists



def load_stopwords(f_path = None):
    stop_words = {}
    f = open(f_path, "r")
    for line in f:
        line = line.strip('\n').lower()
        stop_words[line] = 1
    return stop_words

def init_modules(data_type='amazon'):
    options = {}
   
    options["is_predicting"] = False

    print ("is_predicting: ", options["is_predicting"], ", data = " + data_type)

    options["is_unicode"] = True
    
    print ("loading...")
    all_data = pickle.load(open("/content/drive/MyDrive/data/data_"+ data_type + "_electronics(2).pkl", "rb"))
    [x_raw_train, x_raw_dev, x_raw_test, old_dic, dic, hfw, i2w, w2i, w2w, w2idf, i2user, user2i, i2item, item2i, user2w, item2w] = all_data

    consts = {}
    consts["num_user"] = len(user2i)
    consts["num_item"] = len(item2i)
    consts["dict_size"] = len(dic)+1
    consts["hfw_dict_size"] = len(hfw)+1
    consts["lvt_dict_size"] = LVT_DICT_SIZE+1
    consts["len_y"] = LEN_Y
    if data_type == "yelp":
        consts["len_y"] = LEN_Y_YELP
        consts["lvt_dict_size"] = LVT_DICT_SIZE_YELP

    consts["dim_x"] = DIM_X
    consts["dim_w"] = DIM_W
    consts["dim_h"] = DIM_H
    consts["dim_r"] = DIM_RATING
    consts["max_len_predict"] = consts["len_y"]
    consts["min_len_predict"] = 1

    consts["batch_size"] = 200
    
    consts["test_batch_size"] = 1
    consts["test_samples"] = 2000


    consts["beam_size"] = 5

    consts["lr"] = 1.

    consts["print_size"] = 2#400 2 for demo

    print ("#train = ", len(x_raw_train), ", #dev = ", len(x_raw_dev), ", #test = ", len(x_raw_test))
    print ("#user = ", consts["num_user"], ", #item = ", consts["num_item"])
    print ("#dic = ", consts["dict_size"], ", #hfw = ", consts["hfw_dict_size"])

    modules = {}
    modules["all_data"] = all_data
    hfw.append('<start>')
    modules["hfw"] = hfw
    w2i['<start>']=len(w2i)
    modules["w2i"] = w2i
    
    modules["i2w"] = i2w
    modules["w2w"] = w2w
    w2w['<start>'] = '<start>'
    modules["u2ws"] = user2w
    modules["i2ws"] = item2w
    modules["w2idf"] = w2idf

    modules["eos_emb"] = modules["w2i"][W_EOS]
    modules["unk_emb"] = modules["w2i"][W_UNK]
    modules["<start>"] = modules["w2i"][W_START]
    
    modules["optimizer"] = "adadelta"
    
    modules["stopwords"] = load_stopwords("/content/drive/MyDrive/data/stopwords_en.txt")

    return modules, consts, options


In [ ]:
modules,consts,options = init_modules()

#now create pipeline for data

In [ ]:
# -*- coding: utf-8 -*-
#pylint: skip-file
import sys
import os
import os.path
import time
from operator import itemgetter
import numpy as np
import pickle
import random
#from commons import *


def totensorf(arg):
    arg = tf.convert_to_tensor(arg,dtype=tf.float32)
    return arg
def totensori(arg):
    arg = tf.convert_to_tensor(arg,dtype=tf.int32)
    return arg
def get_batch_index(n, batch_size, is_predict = False):
    i_list = [i for i in range(n)]
    if not is_predict:
        random.shuffle(i_list)

    batch_index = []
    x = []
    for i in range(n):
        x.append(i_list[i])
        if len(x) == batch_size or i == (n-1):
            batch_index.append(x)
            x = []
    return batch_index

def get_batch_data(raw_x, consts, options, modules):
    batch_size = len(raw_x)
    hfw = modules["hfw"]
    w2i = modules["w2i"]
    i2w = modules["i2w"]
    w2w = modules["w2w"]
    w2idf = modules["w2idf"]
    u2ws = modules["u2ws"]
    i2ws = modules["i2ws"]
    len_y = consts["len_y"]
    lvt_dict_size = consts["lvt_dict_size"]

    xu = np.zeros(batch_size)
    xi = np.zeros(batch_size)
    yr = np.zeros(batch_size)
    yr_vec = np.zeros((batch_size, DIM_RATING) )
    y_review_mark = np.zeros((batch_size, lvt_dict_size))
    y_review_tf = np.zeros((batch_size, lvt_dict_size) )
    y_summary_index = np.zeros((len_y+1, batch_size, 1) ) #add <start> token
    y_summary_mark = np.zeros((len_y+1, batch_size) )
    y_summary_lvt = np.zeros((len_y+1, batch_size, 1) )

    dic_review_words = {}
    w2i_review_words = {}
    lst_review_words = []
    lvt_user_item = {w2i[W_EOS]}
    lvt_i2i = {}
    for i in range(batch_size):
        uid, iid, r, summary, review, _, _ = raw_x[i] 
        xu[i] = uid
        xi[i] = iid
        yr[i] = r
        yr_vec[i, int(r)] = 1.
        lvt_user_item |= u2ws[uid] | i2ws[iid]

        if len(summary) > len_y:
            summary = summary[0:len_y-1] + [W_EOS]
        summary = [W_START] + summary
        for wi in range(len(summary)):
            w = summary[wi]
            w = w2w[w]
            if w in dic_review_words:
                dic_review_words[w] += 1
            else:
                w2i_review_words[w] = len(dic_review_words)
                dic_review_words[w] = 1
                lst_review_words.append(w2i[w])
            y_summary_index[wi, i, 0] = w2i[w]
            y_summary_lvt[wi, i, 0] = w2i_review_words[w]
            y_summary_mark[wi, i] = 1

        for w in review:
            w = w2w[w]
            if w in dic_review_words:
                dic_review_words[w] += 1
            else:
                w2i_review_words[w] = len(dic_review_words)
                dic_review_words[w] = 1
                lst_review_words.append(w2i[w])
            y_review_mark[i, w2i_review_words[w]] = 1
            y_review_tf[i, w2i_review_words[w]] += 1#w2idf[w]

    y_review_tf /= 10.

    if len(dic_review_words) < lvt_dict_size:
        for rd_hfw in hfw:
            if rd_hfw not in dic_review_words:
                w2i_review_words[rd_hfw] = len(dic_review_words)
                dic_review_words[rd_hfw] = 0
                lst_review_words.append(w2i[rd_hfw])
            if len(dic_review_words) == lvt_dict_size:
                break
    else:
        print ("!!!!!!!!!!!!")

    for i in range(len(lst_review_words)):
        lvt_i2i[i] = lst_review_words[i]
    assert len(dic_review_words) == lvt_dict_size
    assert len(lst_review_words) == lvt_dict_size

    if options["is_predicting"]:
        lvt_i2i = {}
        lst_review_words = list(lvt_user_item)
        for i in range(len(lst_review_words)):
            lvt_i2i[i] = lst_review_words[i]
        
    lst_review_words = np.asarray(lst_review_words)
    #print(lvt_i2i)
    #print(dic_review_words["adf"])
    xu = totensori(xu)
    xi = totensori(xi)
    yr = totensorf(yr)
    yr_vec = totensorf(yr_vec)
    y_review_mark = totensorf(y_review_mark)
    y_review_tf = totensorf(y_review_tf)
    y_summary_index = totensori(y_summary_index)
    y_summary_mark = totensorf(y_summary_mark)
    y_summary_lvt = totensori(y_summary_lvt)

    return xu, xi, yr, yr_vec, lst_review_words, y_review_tf, y_review_mark, y_summary_index, y_summary_mark, y_summary_lvt, lvt_i2i




# Model

In [ ]:

class RatingUser(tf.keras.layers.Layer):
  def __init__(self,dim_h):
    super(RatingUser,self).__init__()
    self.dense1 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
    self.dense2 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
    self.dense3 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
  def call(self,x_emb,training=False):
    hru = self.dense1(x_emb)
    hru = self.dense2(hru)
    hru = self.dense3(hru)
    return hru


class RatingItem(tf.keras.layers.Layer):
  def __init__(self,dim_h):
    super(RatingItem,self).__init__()
    self.dense1 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
    self.dense2 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
    self.dense3 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
  def call(self,i_emb,training=False):
    hri = self.dense1(i_emb)
    hri = self.dense2(hri)
    hri = self.dense3(hri)
    return hri


class Rating(tf.keras.layers.Layer):
  def __init__(self,dim_h):
    super(Rating,self).__init__()
    self.userdense = tf.keras.layers.Dense(dim_h)
    self.itemdense = tf.keras.layers.Dense(dim_h)
    self.useritemdense = tf.keras.layers.Dense(dim_h)
    self.sigmoid1 = tf.keras.activations.sigmoid
    #self.dense1 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
    self.dense2 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
    self.dense3 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
  def call(self,x_emb,i_emb,training=False):
    hru = self.userdense(x_emb)
    hri = self.itemdense(i_emb)
    hrui = self.useritemdense(x_emb*i_emb)
    hr = self.sigmoid1(hru+hri+hrui)

    hr = self.dense2(hr)
    hr = self.dense3(hr)
    return hr

class RatingUserItem(tf.keras.layers.Layer):
  def __init__(self,dim_h):
    super(RatingUserItem,self).__init__()
    self.userdense = tf.keras.layers.Dense(dim_h)
    self.itemdense = tf.keras.layers.Dense(dim_h)
    self.sigmoid1 = tf.keras.activations.sigmoid
    self.dense2 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
    self.dense3 = tf.keras.layers.Dense(dim_h,activation = 'sigmoid')
  def call(self,x_emb,i_emb,training=False):
    hru = self.userdense(x_emb)
    hri = self.itemdense(i_emb)
    hrui = hru + hri
    hrui = self.sigmoid1(hrui)
    hrui = self.dense2(hrui)   
    hrui = self.dense3(hrui)
    return hrui



class NRTrating(tf.keras.Model):
  def __init__(self):
    super(NRTrating,self).__init__()
    self.num_user = consts['num_user']
    self.num_item = consts['num_item']
    self.dim_x  = consts['dim_x']
    self.dim_h = consts['dim_h']
    self.dim_r = consts['dim_r']
    self.dict_size = consts['hfw_dict_size']
    self.lvt_dict_size = consts['lvt_dict_size']
    
    self.embeddingu = tf.keras.layers.Embedding(self.num_user, self.dim_x)
    self.embeddingi = tf.keras.layers.Embedding(self.num_item, self.dim_x)
    
    self.ratinguser = RatingUser(self.dim_h)

    self.ratingitem = RatingItem(self.dim_h)

    self.ratinguseritem = RatingUserItem(self.dim_h)

    self.rating = Rating(self.dim_h)

    self.whru3 = tf.keras.layers.Dense(self.dim_h)
    self.whri3 = tf.keras.layers.Dense(self.dim_h)
    self.whr3 = tf.keras.layers.Dense(self.dim_h)
    self.sigmoid1 = tf.keras.activations.sigmoid
    self.wrhr = tf.keras.layers.Dense(1)
    self.wembu = tf.keras.layers.Dense(1)
    self.wembi = tf.keras.layers.Dense(1)
    self.wreviewi = tf.keras.layers.Dense(self.lvt_dict_size)
    self.wreviewu = tf.keras.layers.Dense(self.lvt_dict_size)
    self.wreview = tf.keras.layers.Dense(self.lvt_dict_size)
    self.softmax1 = tf.keras.activations.softmax
    self.softmax2 = tf.keras.activations.softmax
    self.softmax3 = tf.keras.activations.softmax
    self.ddense1 = tf.keras.layers.Dense(self.dim_h)
    self.ddense2 = tf.keras.layers.Dense(self.dim_h)
    self.ddense3 = tf.keras.layers.Dense(self.dim_h)
    self.ddense4 = tf.keras.layers.Dense(self.dim_h)
    self.ddense5 = tf.keras.layers.Dense(self.dim_h)
    self.ddense6 = tf.keras.layers.Dense(self.dim_h)
    self.dtanh1 = tf.keras.activations.tanh


  def call(self,XU,XI,emb_r,training=False):
    x_emb = self.embeddingu(XU) #batch_size, dim_x
    i_emb = self.embeddingi(XI)

    hru3 = self.ratinguser(x_emb)
    hri3 = self.ratingitem(i_emb)
    hrui3 = self.ratinguseritem(x_emb,i_emb)
    hr3 = self.rating(x_emb,i_emb)

    hr4 = self.whru3(hru3)+self.whri3(hri3)+self.whr3(hr3)
    hr4 = self.sigmoid1(hr4)

    rating = self.wrhr(hr4)+self.wembu(x_emb)+self.wembi(i_emb)

    review_i = self.wreviewi(hri3)
    review_i = self.softmax1(review_i)
    review_u = self.wreviewu(hru3)
    review_u = self.softmax2(review_u)
    review = self.wreview(hrui3)
    review = self.softmax3(review)
    h_s = self.dtanh1(self.ddense1(x_emb)+self.ddense2(i_emb)+self.ddense3(emb_r)/
                     self.ddense4(hru3)+self.ddense5(hri3)+self.ddense6(hrui3))
    return rating,review,review_i,review_u,x_emb,i_emb,hru3,hri3,hrui3,h_s


class NRTdecoder(tf.keras.Model):
  def __init__(self):
    super(NRTdecoder,self).__init__()
    self.num_user = consts['num_user']
    self.num_item = consts['num_item']
    self.dim_x  = consts['dim_x']
    self.dim_h = consts['dim_h']
    self.dim_r = consts['dim_r']
    self.dict_size = consts['hfw_dict_size']
    self.lvt_dict_size = consts['lvt_dict_size']
    self.dim_w = consts['dim_w']

    self.embedding1 = tf.keras.layers.Embedding(self.dict_size,self.dim_w)
    self.gru = tf.keras.layers.GRU(self.dim_h,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    
    self.dense1 = tf.keras.layers.Dense(self.dict_size)
    self.dense2 = tf.keras.layers.Dense(self.dim_h)
    self.dense3 = tf.keras.layers.Dense(self.dim_h)
    self.dense4 = tf.keras.layers.Dense(self.dim_h)
    self.dense5 = tf.keras.layers.Dense(self.dim_h)
    self.dense6 = tf.keras.layers.Dense(self.dim_h)
    self.tanh = tf.keras.activations.tanh
    self.softmax = tf.keras.activations.softmax
  def call(self,x,hidden,xu_emb,xi_emb,emb_r,training=False):
    
    y_emb = self.embedding1(x)

    x  = tf.concat([tf.expand_dims(hidden, 1), y_emb], axis=-1)
    #print(x.shape)
    output,state = self.gru(x)

    output = tf.reshape(output,(-1,output.shape[2]))
    
    #x=tf.reshape(x,[x.shape[0],x.shape[2]])
    #print(self.dense3(x).shape)
    #x = self.tanh(self.dense2(output)+self.dense3(x)+self.dense4(xu_emb)+
                               #self.dense5(xi_emb)+self.dense6(emb_r))
    #print(x.shape)
    x = self.dense1(output)
    #x = self.dense1(output)

    return x,state



In [ ]:
nrtrating = NRTrating()
nrtdecoder = NRTdecoder()
#nrtrating(tf.random.uniform((200,)),tf.random.uniform((200,)),tf.random.uniform((200,6)))
#nrtdecoder(tf.random.uniform((200,1)),tf.random.uniform((200,20)),tf.random.uniform((200,40)),tf.random.uniform((200,40)),tf.random.uniform((200,4)))[0].shape

In [2]:
def beam_decode(result_paths, fname, batch_raw, modules, consts, options):
    
    print (fname)

    [SUMM_PATH_IDX, SUMM_PATH_WORD, MODEL_PATH_IDX, MODEL_PATH_WORD, RATING_PATH] = result_paths
    beam_size = consts["beam_size"]
    num_live = 1
    num_dead = 0
    samples = []
    sample_scores = np.zeros(beam_size)

    last_traces = [[]]
    last_scores = np.zeros(1)
    last_states = []

    XU, XI, Y_r, Y_r_vec, \
            Dict_lvt, Y_rev_tf, Y_rev_mark, \
            Y_sum_idx, Y_sum_mark, Y_sum_lvt, lvt_i2i = get_batch_data(batch_raw, consts, options, modules)
    

    rat_pred, dec_state, emb_r, xu_emb, xi_emb = nrtrating(XU, XI, consts["test_batch_size"])
    next_y = -np.ones((1, num_live, 1), dtype="int64")
    
    write_rating("".join((RATING_PATH, "rating.", fname)), str(rat_pred[0]) + "\t" + str(Y_r[0]))

    for step in range(consts["max_len_predict"]):
        model.decode_once
        tile_emb_r = np.tile(emb_r, (num_live, 1))
        tile_xu_emb = np.tile(xu_emb, (num_live, 1))
        tile_xi_emb = np.tile(xi_emb, (num_live, 1))

        y_pred, dec_state = nrtdecoder(next_y, dec_state, tile_emb_r, tile_xu_emb, tile_xi_emb, num_live, Dict_lvt)
        
        dict_size = y_pred.shape[-1]
        cand_scores = last_scores + np.log(y_pred) # 分数最大越好
        cand_scores = cand_scores.flatten()
        idx_top_joint_scores = np.argsort(cand_scores)[-(beam_size - num_dead):]
        idx_last_traces = idx_top_joint_scores // dict_size
        idx_word_now = idx_top_joint_scores % dict_size
        top_joint_scores = cand_scores[idx_top_joint_scores]

        traces_now = []
        scores_now = np.zeros((beam_size - num_dead))
        states_now = []
        for i, [j, k] in enumerate(zip(idx_last_traces, idx_word_now)):
            traces_now.append(last_traces[j] + [lvt_i2i[k]])
            scores_now[i] = copy.copy(top_joint_scores[i])
            states_now.append(copy.copy(dec_state[j, :]))

        num_live = 0
        last_traces = []
        last_scores = []
        last_states = []

        for i in range(len(traces_now)):
            if traces_now[i][-1] == modules["eos_emb"]:
                samples.append([str(e) for e in traces_now[i][:-1]])
                sample_scores[num_dead] = scores_now[i]
                num_dead += 1
            else:
                last_traces.append(traces_now[i])
                last_scores.append(scores_now[i])
                last_states.append(states_now[i])
                num_live += 1
        if num_live == 0 or num_dead >= beam_size:
            break

        last_scores = np.array(last_scores).reshape((num_live, 1))
        next_y = np.array([e[-1] for e in last_traces], dtype = "int64").reshape((1, num_live, 1))
        dec_state = np.array(last_states).reshape((num_live, dec_state.shape[-1]))
        assert num_live + num_dead == beam_size

    if num_live > 0:
        for i in range(num_live):
            samples.append([str(e) for e in last_traces[i]])
            sample_scores[num_dead] = last_scores[i]
            num_dead += 1
    
    
    #weight by length
    for i in range(len(sample_scores)):
        sent_len = float(len(samples[i]))
        #sample_scores[i] = sample_scores[i] * math.exp(-sent_len / 8)
        # Google's Neural Machine Translation System
        lpy = math.pow((consts["min_len_predict"] + sent_len), 0.6) / math.pow((consts["min_len_predict"] + 1), 0.6)
        sample_scores[i] = sample_scores[i]  / lpy

    idx_sorted_scores = np.argsort(sample_scores) # 低分到高分
    ly = int(Y_sum_mark[:, 0].sum())
    y_true = Y_sum_idx[0 : ly, 0, 0].tolist()
    y_true = [str(i) for i in y_true[:-1]] # delete <eos>

    sorted_samples = []
    sorted_scores = []
    filter_idx = []
    for e in idx_sorted_scores:
        if len(samples[e]) >= consts["min_len_predict"] and (str(modules["unk_emb"]) not in set(samples[e])):
            filter_idx.append(e)
    if len(filter_idx) == 0:
        filter_idx = idx_sorted_scores
    for e in filter_idx:
        sorted_samples.append(samples[e])
        sorted_scores.append(sample_scores[e])

    num_samples = len(sorted_samples)
    if len(sorted_samples) == 1:
        sorted_samples = sorted_samples[0]
        num_samples = 1
    
    try:
        if beam_size == 1:
            write_summ("".join((SUMM_PATH_IDX, "summ.", fname)), sorted_samples, 1)
        else:
            write_summ("".join((SUMM_PATH_IDX, "summ.", fname)), sorted_samples[-1], 1)
	    
        write_summ("".join((MODEL_PATH_IDX, "model.", fname)), y_true, 1)

        write_summ("".join((SUMM_PATH_WORD, "summ.", fname)), sorted_samples, num_samples, modules["i2w"], sorted_scores)
	    
        write_summ("".join((MODEL_PATH_WORD, "model.", fname)), y_true, 1, modules["i2w"])
    except Exception as e:
        print ("error", fname)
        raise


def predict(data_type, model, modules, consts, options):
    predict_rmse(data_type,  modules, consts, options)
    predict_summary(data_type,  modules, consts, options)


def predict_summary(data_type, model, modules, consts, options):
    print ("start predicting summary ...")
    SUMM_PATH_IDX = SUMM_PATH + data_type + "/sum_idx/"
    SUMM_PATH_WORD = SUMM_PATH + data_type + "/sum_word/"
    MODEL_PATH_IDX = SUMM_PATH + data_type + "/model_idx/"
    MODEL_PATH_WORD = SUMM_PATH + data_type + "/model_word/"
    RATING_PATH = SUMM_PATH + data_type + "/rating/"
    
    if not exists(SUMM_PATH_IDX):
        makedirs(SUMM_PATH_IDX)
    if not exists(SUMM_PATH_WORD):
        makedirs(SUMM_PATH_WORD)
    if not exists(MODEL_PATH_IDX):
        makedirs(MODEL_PATH_IDX)
    if not exists(MODEL_PATH_WORD):
        makedirs(MODEL_PATH_WORD)
    if not exists(RATING_PATH):
        makedirs(RATING_PATH)
    
    rebuild_dir(SUMM_PATH_IDX)
    rebuild_dir(SUMM_PATH_WORD)
    rebuild_dir(MODEL_PATH_IDX)
    rebuild_dir(MODEL_PATH_WORD)
    rebuild_dir(RATING_PATH)

    result_paths = [SUMM_PATH_IDX, SUMM_PATH_WORD, MODEL_PATH_IDX, MODEL_PATH_WORD, RATING_PATH]


    [x_raw_train, x_raw_dev, x_raw_test, old_dic, dic, hfw, i2w, w2i, w2w, w2idf, i2user, user2i, i2item, item2i, user2w, item2w] = modules["all_data"]
    batch_list = datar.get_batch_index(len(x_raw_dev), consts["test_batch_size"], options["is_predicting"])
    error_i = 0
    for batch_index in batch_list:
        local_batch_size = len(batch_index)
        batch_raw = [x_raw_dev[bxi] for bxi in batch_index]
        beam_decode(result_paths, str(batch_index[0]), batch_raw, modules, consts, options)
        error_i += 1
        if error_i == consts["test_samples"]:
            break

def predict_rmse(data_type, modules, consts, options):
    print ("start predicting rmse...")

    [x_raw_train, x_raw_dev, x_raw_test, old_dic, dic, hfw, i2w, w2i, w2w, w2idf, i2user, user2i, i2item, item2i, user2w, item2w] = modules["all_data"]
    batch_list = datar.get_batch_index(len(x_raw_dev), consts["batch_size"])
   
    ae = 0
    se = 0
    se_i = 0
    
    for batch_index in batch_list:
        local_batch_size = len(batch_index)
        batch_raw = [x_raw_dev[bxi] for bxi in batch_index]
        XU, XI, Y_r, Y_r_vec, \
            Dict_lvt, Y_rev_tf, Y_rev_mark, \
            Y_sum_idx, Y_sum_mark, Y_sum_lvt, lvt_i2i = get_batch_data(batch_raw, consts, options, modules)

        rat_pred, dec_state, emb_r, xu_emb, xi_emb = nrtrating(XU, XI, Y_r_vec)

        for i in range(len(batch_raw)):
            se += (rat_pred[i, 0] -  Y_r[i]) * (rat_pred[i, 0] -  Y_r[i])
            ae += abs(rat_pred[i, 0] -  Y_r[i])
        se_i += len(batch_raw)
        
        #print "MAE now = ", ae / se_i,
        #print "RMSE now = ", np.sqrt(se / se_i), 
        #print "#X = ", se_i

    print ("MAE = ", ae / se_i,)
    print ("RMSE = ", np.sqrt(se / se_i), )
    print ("#X = ", se_i)


In [ ]:

optimizer = tf.keras.optimizers.Adam()

def cost_mse(pred,label):
  return tf.reduce_sum(tf.keras.losses.mean_squared_error(label,pred))
def cost_mae(pred,label):
  return tf.reduce_sum(tf.keras.losses.mean_absolute_error(label,pred))

def cost_nll(pred,label,mark):
  cost = -tf.math.log(pred)*label*mark
  cost = tf.reduce_sum(tf.reduce_sum(cost,axis=1))
  return cost

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=nrtrating,
                                 decoder=nrtdecoder)

@tf.function
def train_step(xu, xi, y_rating, y_rat_vec, lvt_dict, y_rev_mark, y_rev_tf, y_sum_idx, y_sum_mark, y_sum_lvt,batch_size,lr):
  loss = 0
  cce = 0
  mse = 0
  nll = 0
  nllu = 0
  nlli = 0
  with tf.GradientTape() as tape:
    rating,review,review_i,review_u,x_emb,i_emb,hru3,hri3,hrui3,dec_hidden = nrtrating(xu,xi,y_rat_vec)
    
    dec_input = tf.expand_dims([modules['<start>']] *y_sum_idx.shape[1] , 1)
    y_sum_idx = tf.transpose(tf.reshape(y_sum_idx,[y_sum_idx.shape[0],y_sum_idx.shape[1]]))
    y_sum_mark = tf.transpose(tf.reshape(y_sum_mark,[y_sum_mark.shape[0],y_sum_mark.shape[1]]))
    print(y_sum_idx.shape,dec_hidden.shape)
    ce = 0
    for t in range(1,y_sum_idx.shape[1]):
      #print(dec_hidden.shape,dec_input.shape)
      predictions,dec_hidden = nrtdecoder(dec_input,dec_hidden,x_emb,i_emb,y_rat_vec)
      #predictions, dec_hidden = nrtdecoder(dec_input,dec_hidden)
      ce+= loss_function(y_sum_idx[:,t],predictions)

      #using teacher forcing
      dec_input = tf.expand_dims(y_sum_idx[:,t],1)
    #y_sum_pred = nrtdecoder(x_emb,i_emb,emb_r,hru3,hri3,hrui3)
    cce += ce/ int(xu.shape[0])
    mse+= cost_mse(rating,y_rating) / int(xu.shape[0])
    nll+= cost_nll(review,y_rev_tf,y_rev_mark) / int(xu.shape[0])
    nllu+= cost_nll(review_u,y_rev_tf,y_rev_mark) / int(xu.shape[0])
    nlli += cost_nll(review_i,y_rev_tf,y_rev_mark)/ int(xu.shape[0])
    loss += cce + mse+nll+nllu+nlli
  batch_loss = (loss)

  variables = nrtrating.trainable_variables + nrtdecoder.trainable_variables
  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return [batch_loss,cce,mse,nll,nllu,nlli],rating

In [ ]:
def run(data_type, existing_model_name = None):
    modules, consts, options = init_modules(data_type)
    
    if options["is_predicting"]:
        need_load_model = True
        training_model = False
    else:
        need_load_model = False
        training_model = True


    print ("compiling...")
    #model = NeuralRecsys(modules, consts, options)
    

    if training_model:
        print ("training......")
        [x_raw_train, x_raw_dev, x_raw_test, old_dic, dic, hfw, i2w, w2i, w2w, w2idf, i2user, user2i, i2item, item2i, user2w, item2w] = modules["all_data"]
        for epoch in range(0, 50):
            
            if epoch > 2:
                consts["lr"] = 0.1

            start = time.time()
            batch_list = get_batch_index(len(x_raw_train), consts["batch_size"]) #consts["batch_size"]
            error = 0
            error_i = 0
            error_rmse = 0
            e_nll = 0
            e_nllu = 0
            e_nlli = 0
            e_cce = 0
            e_mae = 0
            e_l2n = 0
            for batch_index in batch_list:
                local_batch_size = len(batch_index)
                batch_raw = [x_raw_train[bxi] for bxi in batch_index]
                load_time = time.time()
                XU, XI, Y_r, Y_r_vec, \
                        Dict_lvt, Y_rev_tf, Y_rev_mark, \
                        Y_sum_idx, Y_sum_mark, Y_sum_lvt, lvt_i2i = get_batch_data(batch_raw, consts, options, modules)
                #print(type(XU))
                #ypad = tf.pad(Y_sum_idx,tf.constant([[1,0],[0,0],[0,0]]),"constant")
                #yupdate = tf.slice(ypad,[0,0,0],[21,-1,-1])
                #Y_sum_idx = yupdate
                #print(Y_sum_idx,Y_sum_mark)
                
                cost,rating = train_step(XU, XI, Y_r, Y_r_vec, \
                                                   Dict_lvt, Y_rev_tf, Y_rev_mark, \
                                                   Y_sum_idx, Y_sum_mark, Y_sum_lvt, \
                                                   local_batch_size, consts["lr"])
                #print cost, mse, nll, cce, l2n
                error += cost[0]
                e_cce += cost[1]
                error_rmse+= cost[2]
                e_nll += cost[3]
                e_nllu += cost[4]
                e_nlli += cost[5]


                #if(error_i==19):
                  #print(Y_r,tf.convert_to_tensor(rating))
                error_i+=1  
                if error_i % 50 == 0:
                  print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,error_i,cost[0].numpy()))
            # saving (checkpoint) the model every 2 epochs
            if (epoch + 1) % 2 == 0:
              checkpoint.save(file_prefix = checkpoint_prefix)
            #print_sent_dec(sum_pred, Y_sum_idx, Y_sum_mark, modules, consts, options, lvt_i2i)
            print ("epoch=", epoch, ", Error = ", error / len(batch_list),)
            print (", RMSE = ", error_rmse / len(batch_list), ", time=", time.time()-start)
            print ("epoch=", epoch, ", Error = ", error / len(batch_list),",cce = ",e_cce/len(batch_list),", mse = ",error_rmse/len(batch_list),", nll =",e_nll/len(batch_list),
                   ", nllu= ",e_nllu/len(batch_list), ", nlli =", e_nlli/len(batch_list))
            #print (", RMSE = ", error_rmse / len(batch_list), ", time=", time.time()-start)
    else:
      # restoring the latest checkpoint in checkpoint_dir
      checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
      predict(data_type, modules, consts, options)
run('amazon')
        

In [ ]:
run('amazon',None)